# Goals added prediction

In [2]:
import pandas as pd
import numpy as np


In [10]:
df_raw = pd.read_csv("data/g+_usd_age.csv", usecols=range(1,25))
df_raw

,player,club,age,join_age,usd,Season,Goals Added,name,Player,Club,...,G,A,SHTS,SOG,GWG,PKG/A,HmG,RdG,G/90min,SC%
0,E. Segura,Los Angeles FC,24.0,22.0,2360000.0,2020.0,0.46,Eddie Segura,Eddie Segura,LAFC,...,2.0,1.0,17.0,5.0,1.0,0/0,2.0,0.0,0.10,11.8
1,M. Traore,Los Angeles FC,18.0,18.0,118000.0,2020.0,-0.11,Mohamed Traore,Mohamed Traore,LAFC,...,0.0,0.0,1.0,0.0,0.0,0/0,0.0,0.0,0.00,0.0
2,D. Palacios,Los Angeles FC,21.0,20.0,1180000.0,2020.0,0.27,Diego Palacios,Diego Palacios,LAFC,...,0.0,1.0,4.0,0.0,0.0,0/0,0.0,0.0,0.00,0.0
3,M. Farfán,Los Angeles FC,22.0,22.0,1180000.0,2020.0,-0.20,Marco Farfan,Marco Farfan,LAFC,...,0.0,0.0,3.0,0.0,0.0,0/0,0.0,0.0,0.00,0.0
4,J. Harvey,Los Angeles FC,37.0,33.0,295000.0,2020.0,-0.52,Jordan Harvey,Jordan Harvey,LAFC,...,0.0,1.0,2.0,0.0,0.0,0/0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,A. Rusnák,Real Salt Lake City,26.0,22.0,2360000.0,2020.0,0.63,Albert Rusnak,Albert Rusnak,RSL,...,3.0,2.0,31.0,8.0,2.0,2/3,3.0,0.0,0.21,9.7
437,J. Meram,Real Salt Lake City,32.0,31.0,1180000.0,2020.0,-0.03,Justin Meram,Justin Meram,RSL,...,3.0,3.0,16.0,8.0,0.0,0/0,2.0,1.0,0.24,18.8
438,T. Schmitt,Real Salt Lake City,23.0,21.0,236000.0,2020.0,-0.50,Tate Schmitt,Tate Schmitt,RSL,...,0.0,0.0,1.0,0.0,0.0,0/0,0.0,0.0,0.00,0.0
439,C. Garcia,Real Salt Lake City,18.0,17.0,118000.0,2020.0,-0.01,Christopher Garcia,Christopher Garcia,RSL,...,0.0,0.0,0.0,0.0,0.0,0/0,0.0,0.0,0.00,0.0


In [11]:
[1:7]

SyntaxError: invalid syntax (<ipython-input-11-1bd16290b2e9>, line 1)

In [ ]:
# combine 